In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import string
from datetime import datetime
import time 
import unicodedata
import pandas as pd
import string
import re

In [60]:
# Read resumes from the site (indeed.com)
def resume_parser(url):

    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page)
    work_summary = []
    edu_summary = []
    Summary,skills,additional_info = [""]*3
    #div_name = soup.findAll('div',{"id" : "basic_info_cell","class":"data_display"})
    #name = div_name[0].findAll('h1',{"id" : "resume-contact","class":"fn"})[0].text
    if len(soup.findAll('div',{ "id" : "work-experience-items" })) > 0 :
        div_work_items= soup.findAll('div',{ "id" : "work-experience-items" })
        div_work = div_work_items[0].findAll('div',{"class" : "work-experience-section "})
        div_work.append(div_work_items[0].findAll('div',{"class" : "work-experience-section last"})[0])
        for i in range(len(div_work)):
            
            if len(div_work[i].findAll('p',{"class":"work_title title"})) > 0:
                work_title = div_work[i].findAll('p',{"class":"work_title title"})[0].text
            else:
                 work_title = ""
            if len(div_work[i].findAll('div',{"class" : "work_company"})) >0:
                work_company = div_work[i].findAll('div',{"class" : "work_company"})[0].text
            else:
                work_company = ""
            if len(div_work[i].findAll('p',{"class":"work_dates"})) > 0 :
                work_dates = div_work[i].findAll('p',{"class":"work_dates"})[0].text
            else:
                work_dates = ""
            if len(div_work[i].findAll('p',{"class":"work_description"})) > 0 :
                work_ex = div_work[i].findAll('p',{"class":"work_description"})[0].text
            else:
                work_ex = ""
            
            work_summary.append([work_title,work_company,work_dates,work_ex])
            
            #print(name,work_title," ",work_company," ",work_dates," ",work_ex,"\n")
            
    if len(soup.findAll('p',{"id" : "res_summary"})) > 0 :
        Summary = soup.findAll('p',{"id" : "res_summary"})[0].text
    
    if len(soup.findAll('div',{ "id" : "education-items" })) > 0 :
        div_edu_items= soup.findAll('div',{ "id" : "education-items" })
        div_edu = div_edu_items[0].findAll('div',{"class" : "education-section "})
        div_edu.append(div_edu_items[0].findAll('div',{"class" : "education-section last"})[0])
        for i in range(len(div_edu)):

            if len(div_edu[i].findAll('p',{"class" : "edu_title"})) > 0 :
                edu_title = edu_title = div_edu[i].findAll('p',{"class" : "edu_title"})[0].text
            else:
                edu_title = ""
            
            if len(div_edu[i].findAll('div',{"class" : "edu_school"})) > 0:
                edu_school= div_edu[i].findAll('div',{"class" : "edu_school"})[0].text
            else:
                edu_school= ""
                
            
            
            edu_summary.append([edu_title,edu_school])
            #print(edu_title," ",edu_school," ",edu_dates,"\n")
    
    if len(soup.findAll('div',{ "class" : "skill-container resume-element" })) > 0:
        skills = soup.findAll('div',{ "class" : "skill-container resume-element" })[0].text
        #print(skills,"\n")
        
    if len(soup.findAll('div',{ "id" : "additionalinfo-section" })) > 0 :
        additional_info = soup.findAll('div',{ "id" : "additionalinfo-section" })[0].text
        #print(additional_info)
    return work_summary,Summary,edu_summary,skills,additional_info
    
    
#work_summary,Summary,edu_summary,skills,additional_info = resume_parser('http://www.indeed.com/r/Nan-Li/98159002e3b9337b')
#print(name,"Work Summary:",work_summary,"\n\n","Summary:",Summary,"\n\n","Education:",edu_summary,"\n\n","Skills:",skills,"\n\n","Additional Info",additional_info)  

In [61]:
# Transform the unicode string 
def str_transform(raw_string):
    translator = str.maketrans({key: None for key in string.punctuation})
    raw_string = raw_string.translate(translator)
    raw_string = unicodedata.normalize("NFKD",raw_string ).encode('ascii', 'ignore').decode("utf-8") 
    raw_string = raw_string.lower()
    raw_string = ' '.join(raw_string.split())
    
    cleaned_string = ''.join([i for i in raw_string if not i.isdigit()])
    return cleaned_string


    

In [62]:
# Clean the text(education and work experience) read from the site
def cleaning(work_summary,Summary,edu_summary,skills,additional_info):
    
    clean_work_summary = []
    clean_edu_summary = []
    translator = str.maketrans({key: None for key in string.punctuation})
    for i in range(len(work_summary)):
        temp_item = work_summary[i]
        for j in range(len(temp_item)):
            temp_item[j] = str_transform(temp_item[j])

        clean_work_summary.append(temp_item)
    for i in range(len(edu_summary)-1):
        temp_item = edu_summary[i]
        for j in range(len(temp_item)-1):
            temp_item[j] = str_transform(temp_item[j])
        clean_edu_summary.append(temp_item)
    Summary = str_transform(Summary)
    skills = str_transform(skills)
    additional_info = str_transform(additional_info)
    return work_summary,Summary,edu_summary,skills,additional_info

    
    

In [214]:

# List of page urls for data consultant
page_url.append("http://www.indeed.com/resumes?q=analytics+consultant&l=")
url = 'http://www.indeed.com/resumes?q=analytics+consultant&co=US&start='
for num in [50,100]:
    page_url.append(url+str(num))


In [221]:
page_url = []
# List of page urls for data analytics project manager
page_url.append("http://www.indeed.com/resumes?q=analytics+project+manager&l=")
url = 'http://www.indeed.com/resumes?q=analytics+project+manager&co=US&start='
for num in [50,100]:
    page_url.append(url+str(num))


In [222]:
# List of page urls for data engineer
page_url.append("http://www.indeed.com/resumes?q=data+engineer&l=")
url = 'http://www.indeed.com/resumes?q=data+engineer&co=US&start='
for num in [50,100]:
    page_url.append(url+str(num))

In [223]:
# List of page urls for data analyst
page_url.append("http://www.indeed.com/resumes?q=data+analyst&l=")
url = 'http://www.indeed.com/resumes?q=data+analyst&co=US&start='
for num in [50,100]:
    page_url.append(url+str(num))

In [224]:
# List of page urls for business data analyst
page_url.append("http://www.indeed.com/resumes?q=business+data+analyst&l=")
url = 'http://www.indeed.com/resumes?q=business+data+analyst&co=US&start='
for num in [50,100]:
    page_url.append(url+str(num))

In [225]:
# List of page urls for business intelligence
page_url.append("http://www.indeed.com/resumes?q=business+intelligence&l=")
url = 'http://www.indeed.com/resumes?q=business+intelligence&co=US&start='
for num in [50,100]:
    page_url.append(url+str(num))
print(len(page_url))
print(page_url)

15
['http://www.indeed.com/resumes?q=analytics+project+manager&l=', 'http://www.indeed.com/resumes?q=analytics+project+manager&co=US&start=50', 'http://www.indeed.com/resumes?q=analytics+project+manager&co=US&start=100', 'http://www.indeed.com/resumes?q=data+engineer&l=', 'http://www.indeed.com/resumes?q=data+engineer&co=US&start=50', 'http://www.indeed.com/resumes?q=data+engineer&co=US&start=100', 'http://www.indeed.com/resumes?q=data+analyst&l=', 'http://www.indeed.com/resumes?q=data+analyst&co=US&start=50', 'http://www.indeed.com/resumes?q=data+analyst&co=US&start=100', 'http://www.indeed.com/resumes?q=business+data+analyst&l=', 'http://www.indeed.com/resumes?q=business+data+analyst&co=US&start=50', 'http://www.indeed.com/resumes?q=business+data+analyst&co=US&start=100', 'http://www.indeed.com/resumes?q=business+intelligence&l=', 'http://www.indeed.com/resumes?q=business+intelligence&co=US&start=50', 'http://www.indeed.com/resumes?q=business+intelligence&co=US&start=100']


In [220]:
with open("links_list.csv",'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(page_url)
print(len(page_url))

21


In [54]:
# Get the list of resume urls 
resumes = []
site_url = "http://www.indeed.com"
for page_index in page_url:
    print(page_index)
    response = requests.get(page_index)
    page = response.text
    soup = BeautifulSoup(page)
    #page = open(page_index)
    #soup = BeautifulSoup(page.read())
    result_list = soup.findAll('ol',{ "class" : "resultsList" })
    results = result_list[0].findAll('div',{ "class" : "clickable_resume_card" })
    
    for i in range(len(results)):
        temp_url = results[i]['onclick'].split()[0]
        temp_url = temp_url.replace("window.open('","")
        url = temp_url[:temp_url.find("?")]
        resumes.append(site_url+url)

import csv
with open("resumes.csv",'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(resumes)
print(len(resumes))

http://www.indeed.com/resumes?q=attorney&l=
0


/Users/Manoj/anaconda/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [57]:
# Scrape the resumes from indeed, clean and save the data in a dataframe
resume_summary = []
count = 0
for res in resumes:
    final_summary = {}
    print(res)
    work_summary,Summary,edu_summary,skills,additional_info = resume_parser(res)
    work_summary,Summary,edu_summary,skills,additional_info = cleaning(work_summary,Summary,edu_summary,skills,additional_info)
    #final_summary["name"] = name
    final_summary["work_summary"] = work_summary
    final_summary["Summary"] = Summary
    final_summary["edu_summary"] = edu_summary
    final_summary["skills"] = skills
    final_summary["additional_info"] = additional_info
    resume_summary.append(final_summary)
    count = count + 1
    if count%100 == 0:
        print("Processed:",count)
#print(resume_summary)
    

In [ ]:
# Connect to MongoDB

In [20]:
from pymongo import MongoClient

In [21]:
client = MongoClient()

In [22]:
client.resumes_cluster.drop_collection('resumes')

In [23]:
resumes = client.resumes_cluster.resumes

In [24]:
for item in resume_summary:
    resumes.insert(item)

/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  from ipykernel import kernelapp as app
